## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [11]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_0_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_exec_csv/"+'gcc6'+"_0_32_exec.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_0_32.head()

(13936744, 2)
reset_index 완료
input data shape


,bin,label
0,53,1
1,83,0
2,ec,0
3,08,0
4,e8,0


In [12]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32['label'].value_counts())

256
0    13892354
1       44390
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [13]:
######################## 
idx = gcc6_0_32[gcc6_0_32['label']==1].index  # 407, 474 ...
ls = list(idx)

# 최종 뽑을 행에 대한 index
ls_idx = [] 
left_idx, right_idx = 0, 32 # 3개씩

# 6gram
for k in range(left_idx, right_idx):
    ls_idx.extend(list(idx + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx = list(filter(lambda x: x<len(gcc6_0_32), ls_idx))
print(len(ls_idx))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub = len(ls_idx)%(right_idx)
print('나머지', sub)

ls_idx = ls_idx[:len(ls_idx)-sub]
print('최종 길이', len(ls_idx))

print('gcc6_0_32', len(ls_idx))

# loc 로 수정필요
gcc6_0_32_Ngram = gcc6_0_32.loc[ls_idx,:].copy()

1420468
나머지 20
최종 길이 1420448
gcc6_0_32 1420448


## (4) false data 만들기

In [14]:
# false data 만들기 - False 데이터 랜덤 생성
# random index
random_idx = np.random.randint(len(gcc6_0_32)-right_idx)

# 목표치
goal = len(gcc6_0_32_Ngram)/right_idx
count=0

d = pd.DataFrame(columns=gcc6_0_32.columns)

# goal 에 도달할 때까지
while count!=goal:
    
    # 진행상황 살펴보기 위함
    if count%1000==0:
        print(count, end=' ')
        
    # 랜덤 N 바이트씩 뽑음
    df = gcc6_0_32[random_idx:random_idx+right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df['label']:
        d = pd.concat([d, df])
        count+=1

print('완료')

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 완료


## (5) False Data + True Data 합치기

In [15]:
ff = pd.DataFrame(columns=gcc6_0_32.columns)

for i in range(0, int(len(d)/right_idx)):
    ff = pd.concat([ff, gcc6_0_32[i*right_idx:(i+1)*right_idx], d[i*right_idx:(i+1)*right_idx]])
    
    if i%1000==0:
        print(i, end=' ')
ff.shape

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 

(2840896, 2)

## (6) one hot encoding

In [16]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot_Ngram = pd.get_dummies(ff['bin'])
gcc6_0_32_onehot_Ngram = pd.concat([ff['label'], gcc6_0_32_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(gcc6_0_32_onehot_Ngram.shape)

원핫인코딩완료
(2840896, 257)


In [17]:
# 훈련 데이터, 훈련 라벨
x_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram.iloc[:,1:].to_numpy()
y_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram['label'].to_numpy()
print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

x_gcc6_0_32_3 = x_gcc6_0_32_3.reshape(-1, right_idx, x_gcc6_0_32_3.shape[1])
y_gcc6_0_32_3 = y_gcc6_0_32_3.reshape(-1, right_idx, 1)

print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

(2840896, 256) (2840896,)
(88778, 32, 256) (88778, 32, 1)


## (7) 모델

In [18]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = Bidirectional(LSTM(32, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 64)            73984     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             65        
Total params: 74,049
Trainable params: 74,049
Non-trainable params: 0
_________________________________________________________________


## (8) 학습 - 10 KFold

In [19]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_0_32_3, y_gcc6_0_32_3):
    print('======Training stage======')
    model1.fit(x_gcc6_0_32_3[train],
               y_gcc6_0_32_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_0_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/10
79900/79900 [==============================] - 52s 648us/step - loss: 0.0073 - accuracy: 0.9985
Epoch 2/10
79900/79900 [==============================] - 51s 638us/step - loss: 0.0026 - accuracy: 0.9992
Epoch 3/10
79900/79900 [==============================] - 51s 641us/step - loss: 0.0023 - accuracy: 0.9993
Epoch 4/10
79900/79900 [==============================] - 51s 641us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 5/10
79900/79900 [==============================] - 51s 640us/step - loss: 0.0021 - accuracy: 0.9994
Epoch 6/10
79900/79900 [==============================] - 47s 583us/step - loss: 0.0019 - accuracy: 0.9994
Epoch 7/10
79900/79900 [==============================] - 42s 528us/step - loss: 0.0018 - accuracy: 0.9994
Epoch 8/10
79900/79900 [==============================] - 42s 527us/step - loss: 0.0018 - accuracy: 0.9994
Epoch 9/10
79900/79900 [==============================] - 42s 527us/step - loss: 0.0017 - accuracy: 0.9995
Epoch 10/1

accuracy_score 0.9994262502815949
recall_score 0.6876640419947506
precision_score 0.8562091503267973
f1_score 0.7627365356622998
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 42s 524us/step - loss: 0.0016 - accuracy: 0.9995
Epoch 2/10
79900/79900 [==============================] - 42s 523us/step - loss: 0.0015 - accuracy: 0.9995
Epoch 3/10
79900/79900 [==============================] - 42s 522us/step - loss: 0.0015 - accuracy: 0.9995
Epoch 4/10
79900/79900 [==============================] - 42s 524us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 5/10
79900/79900 [==============================] - 42s 523us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 6/10
79900/79900 [==============================] - 42s 523us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 7/10
79900/79900 [==============================] - 42s 523us/step - loss: 0.0012 - accuracy: 0.9996
Epoch 8/10
79900/79900 [==============================] - 42s 523us/step - loss: 0.0012 - accur

accuracy_score 0.9994297702185176
recall_score 0.7592592592592593
precision_score 0.7454545454545455
f1_score 0.7522935779816514
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 42s 522us/step - loss: 0.0011 - accuracy: 0.9996
Epoch 2/10
79900/79900 [==============================] - 42s 522us/step - loss: 0.0010 - accuracy: 0.9997
Epoch 3/10
79900/79900 [==============================] - 42s 522us/step - loss: 9.8858e-04 - accuracy: 0.9997
Epoch 4/10
79900/79900 [==============================] - 42s 528us/step - loss: 9.2359e-04 - accuracy: 0.9997
Epoch 5/10
79900/79900 [==============================] - 42s 521us/step - loss: 8.6866e-04 - accuracy: 0.9997
Epoch 6/10
79900/79900 [==============================] - 42s 525us/step - loss: 8.2553e-04 - accuracy: 0.9997
Epoch 7/10
79900/79900 [==============================] - 42s 530us/step - loss: 7.8864e-04 - accuracy: 0.9998
Epoch 8/10
79900/79900 [==============================] - 43s 535us/step - 

accuracy_score 0.9995036888938951
recall_score 0.7630769230769231
precision_score 0.7948717948717948
f1_score 0.7786499215070644
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 43s 536us/step - loss: 7.7205e-04 - accuracy: 0.9998
Epoch 2/10
79900/79900 [==============================] - 43s 537us/step - loss: 7.0237e-04 - accuracy: 0.9998
Epoch 3/10
79900/79900 [==============================] - 43s 537us/step - loss: 6.5809e-04 - accuracy: 0.9998
Epoch 4/10
79900/79900 [==============================] - 43s 537us/step - loss: 5.7479e-04 - accuracy: 0.9998
Epoch 5/10
79900/79900 [==============================] - 43s 537us/step - loss: 5.6560e-04 - accuracy: 0.9998
Epoch 6/10
79900/79900 [==============================] - 43s 536us/step - loss: 5.0651e-04 - accuracy: 0.9998
Epoch 7/10
79900/79900 [==============================] - 43s 537us/step - loss: 4.7638e-04 - accuracy: 0.9999
Epoch 8/10
79900/79900 [==============================] - 43s 536us

accuracy_score 0.9996832056769542
recall_score 0.8702702702702703
precision_score 0.8846153846153846
f1_score 0.8773841961852861
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 43s 537us/step - loss: 5.0302e-04 - accuracy: 0.9999
Epoch 2/10
79900/79900 [==============================] - 43s 537us/step - loss: 4.3009e-04 - accuracy: 0.9999
Epoch 3/10
79900/79900 [==============================] - 43s 537us/step - loss: 3.8602e-04 - accuracy: 0.9999
Epoch 4/10
79900/79900 [==============================] - 43s 536us/step - loss: 3.6766e-04 - accuracy: 0.9999
Epoch 5/10
79900/79900 [==============================] - 43s 537us/step - loss: 3.3019e-04 - accuracy: 0.9999
Epoch 6/10
79900/79900 [==============================] - 43s 538us/step - loss: 2.8088e-04 - accuracy: 0.9999
Epoch 7/10
79900/79900 [==============================] - 43s 538us/step - loss: 2.8324e-04 - accuracy: 0.9999
Epoch 8/10
79900/79900 [==============================] - 43s 538us

...

accuracy_score 0.9997888037846362
recall_score 0.9298245614035088
precision_score 0.8983050847457628
f1_score 0.9137931034482759
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 43s 537us/step - loss: 2.3756e-04 - accuracy: 0.9999
Epoch 2/10
79900/79900 [==============================] - 43s 536us/step - loss: 1.8349e-04 - accuracy: 1.0000
Epoch 3/10
79900/79900 [==============================] - 43s 535us/step - loss: 1.4896e-04 - accuracy: 1.0000
Epoch 4/10
79900/79900 [==============================] - 43s 535us/step - loss: 1.4317e-04 - accuracy: 1.0000
Epoch 5/10
79900/79900 [==============================] - 43s 536us/step - loss: 1.1771e-04 - accuracy: 1.0000
Epoch 6/10
79900/79900 [==============================] - 43s 537us/step - loss: 1.2595e-04 - accuracy: 1.0000
Epoch 7/10
79900/79900 [==============================] - 43s 535us/step - loss: 9.6597e-05 - accuracy: 1.0000
Epoch 8/10
79900/79900 [==============================] - 43s 535us

accuracy_score 0.9998768022077045
recall_score 0.9371428571428572
precision_score 0.9618768328445748
f1_score 0.9493487698986975
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 43s 533us/step - loss: 1.5997e-04 - accuracy: 1.0000
Epoch 2/10
79900/79900 [==============================] - 43s 535us/step - loss: 1.2785e-04 - accuracy: 1.0000
Epoch 3/10
79900/79900 [==============================] - 43s 534us/step - loss: 9.1953e-05 - accuracy: 1.0000
Epoch 4/10
79900/79900 [==============================] - 43s 533us/step - loss: 1.0451e-04 - accuracy: 1.0000
Epoch 5/10
79900/79900 [==============================] - 43s 533us/step - loss: 1.0240e-04 - accuracy: 1.0000
Epoch 6/10
79900/79900 [==============================] - 43s 534us/step - loss: 6.5589e-05 - accuracy: 1.0000
Epoch 7/10
79900/79900 [==============================] - 43s 535us/step - loss: 7.4051e-05 - accuracy: 1.0000
Epoch 8/10
79900/79900 [==============================] - 44s 547us

accuracy_score 0.9998873620184726
recall_score 0.936046511627907
precision_score 0.9698795180722891
f1_score 0.9526627218934911
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 47s 587us/step - loss: 1.1879e-04 - accuracy: 1.0000
Epoch 2/10
79901/79901 [==============================] - 48s 605us/step - loss: 1.0302e-04 - accuracy: 1.0000
Epoch 3/10
79901/79901 [==============================] - 48s 607us/step - loss: 6.0072e-05 - accuracy: 1.0000
Epoch 4/10
79901/79901 [==============================] - 49s 609us/step - loss: 8.3748e-05 - accuracy: 1.0000
Epoch 5/10
79901/79901 [==============================] - 49s 609us/step - loss: 6.5524e-05 - accuracy: 1.0000
Epoch 6/10
79901/79901 [==============================] - 48s 605us/step - loss: 4.8131e-05 - accuracy: 1.0000
Epoch 7/10
79901/79901 [==============================] - 49s 611us/step - loss: 5.1028e-05 - accuracy: 1.0000
Epoch 8/10
79901/79901 [==============================] - 49s 612us/

accuracy_score 0.9999331136645263
recall_score 0.9654178674351584
precision_score 0.97953216374269
f1_score 0.9724238026124817
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 49s 611us/step - loss: 1.2752e-04 - accuracy: 1.0000
Epoch 2/10
79901/79901 [==============================] - 49s 611us/step - loss: 9.8079e-05 - accuracy: 1.0000
Epoch 3/10
79901/79901 [==============================] - 49s 611us/step - loss: 7.2137e-05 - accuracy: 1.0000
Epoch 4/10
79901/79901 [==============================] - 49s 615us/step - loss: 3.6033e-05 - accuracy: 1.0000
Epoch 5/10
79901/79901 [==============================] - 49s 612us/step - loss: 3.8452e-05 - accuracy: 1.0000
Epoch 6/10
79901/79901 [==============================] - 49s 615us/step - loss: 7.9393e-05 - accuracy: 1.0000
Epoch 7/10
79901/79901 [==============================] - 49s 617us/step - loss: 7.9690e-05 - accuracy: 1.0000s - loss: 7.9920e-05 - accuracy: 1.00 - ETA: 0s - loss: 7.9823e
Epoch 

accuracy_score 0.9999401543314183
recall_score 0.9648093841642229
precision_score 0.9850299401197605
f1_score 0.9748148148148148

K-fold cross validation Accuracy: [0.9994262502815949, 0.9994297702185176, 0.9995036888938951, 0.9996832056769542, 0.9996902455507998, 0.9997888037846362, 0.9998768022077045, 0.9998873620184726, 0.9999331136645263, 0.9999401543314183]

K-fold cross validation Recall: [0.6876640419947506, 0.7592592592592593, 0.7630769230769231, 0.8702702702702703, 0.8413597733711048, 0.9298245614035088, 0.9371428571428572, 0.936046511627907, 0.9654178674351584, 0.9648093841642229]

K-fold cross validation Precision: [0.8562091503267973, 0.7454545454545455, 0.7948717948717948, 0.8846153846153846, 0.9027355623100304, 0.8983050847457628, 0.9618768328445748, 0.9698795180722891, 0.97953216374269, 0.9850299401197605]

K-fold cross validation F1-Score: [0.7627365356622998, 0.7522935779816514, 0.7786499215070644, 0.8773841961852861, 0.8709677419354839, 0.9137931034482759, 0.949348769

## (9) 평가지표

In [20]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997159396628519
10-Fold Cross_validation. Recall : 0.8654871449745963
10-Fold Cross_validation. Precision : 0.8978509977103629
10-Fold Cross_validation. F1-Score : 0.8805075185939545
